In [1]:
import numpy as np
import pandas as pd
#col_array = ['label', 'label2' 'label_id', 'speed', 'heading_x', 'heading_y', 'heading_z', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z', 'pressure_hPa', 'log_timestamp']
col_array = ['label_id', 'acceleration_x', 'acceleration_y', 'acceleration_z', 'gyroscope_x', 'gyroscope_y', 'gyroscope_z', 'magnetic_field_x', 'magnetic_field_y', 'magnetic_field_z']
path = "../posmo/experiments/data/posmo_segments/v8/" 
def read_values(csv, col_array, path="../posmo/experiments/data/posmo_segments/v8/"):
    return pd.read_csv(path + csv).loc[:, col_array] 

In [2]:
import tensorflow as tf

In [3]:
import glob
files = []
for fname in glob.glob("../posmo/experiments/data/posmo_segments/v8/*.csv"):
    files.append(fname)
for file in files:
    if file in glob.glob("../posmo/experiments/data/posmo_segments/v8/Other*"):
        files.remove(file) 

In [4]:
def feature_extraction(df, f_train, l_train, f_test, l_test, seq_len=250):
    df_np = df.as_matrix()
    df_np = df_np[:(df_np.shape[0] - df_np.shape[0] % seq_len), :]
    df_np = df_np.reshape(int(df_np.shape[0]/seq_len), seq_len, df_np.shape[1])
    for i in range(df_np.shape[0]):
        label = df_np[i, 0, 0]
        feature = np.expand_dims(df_np[i, :, 1:10], axis=0)
        if i % 10 == 0:
            l_test = np.append(l_test, label)
            f_test = np.concatenate([f_test, feature], axis=0)
        else:
            l_train = np.append(l_train, label)
            f_train = np.concatenate([f_train, feature], axis=0)
    #feature = df_np[:, :, 1:18]
    #len = feature.shape[0]
    #n = int((len - len % 10) / 10)
    #feature_test = df_np[:n, :, 1:18]
    #feature_train = df_np[n:, :, 1:18]
    #f_test = np.concatenate([f_test, feature_test], axis=0)
    #f_train = np.concatenate([f_train, feature_train], axis=0)
    return f_train, l_train, f_test, l_test

In [5]:
def return_shapes():
    return f_train.shape, l_train.shape, f_test.shape, l_test.shape

In [6]:
f_train = np.empty([1, 250, 9])
l_train = np.array([])
f_test = np.empty([1, 250, 9])
l_test = np.array([])

In [7]:
for file in files:
    df = read_values(file, col_array, path="")
    f_train, l_train, f_test, l_test = feature_extraction(df, f_train, l_train, f_test, l_test)

In [8]:
f_train = f_train[1:, :, :]
f_test = f_test[1:, :, :]

In [9]:
return_shapes()

((13255, 250, 9), (13255,), (1523, 250, 9), (1523,))

In [10]:
l_train_one_hot = tf.one_hot(l_train, 14)

In [11]:
l_test_one_hot = tf.one_hot(l_test, 14)

In [12]:
def neural_net(activation_function, conv_layers, learning_rate, epochs, dropout, n_filters, seed):
    # network Parameters
    n_classes = 14

    tf.set_random_seed(seed)

    if conv_layers == 2:
        weights = {
            'wc1': tf.Variable(tf.random_normal([n_filters, 9, 32])),
            'wc2': tf.Variable(tf.random_normal([n_filters, 32, 64])),
            'wc3': tf.Variable(tf.random_normal([n_filters, 64, 128])),
            'wd1': tf.Variable(tf.random_normal([10*64, 1024])),
            'out': tf.Variable(tf.random_normal([1024, n_classes]))}
    elif conv_layers == 3:
        weights = {
            'wc1': tf.Variable(tf.random_normal([n_filters, 9, 32])),
            'wc2': tf.Variable(tf.random_normal([n_filters, 32, 64])),
            'wc3': tf.Variable(tf.random_normal([n_filters, 64, 128])),
            'wd1': tf.Variable(tf.random_normal([10*128, 1024])),
            'out': tf.Variable(tf.random_normal([1024, n_classes]))}

    biases = {
        'bc1': tf.Variable(tf.random_normal([32])),
        'bc2': tf.Variable(tf.random_normal([64])),
        'bc3': tf.Variable(tf.random_normal([128])),
        'bd1': tf.Variable(tf.random_normal([1024])),
        'out': tf.Variable(tf.random_normal([n_classes]))}

    # changed relu to sigmoid
    # (best) result from reLu 
    def conv1d(x, W, b, s):
        x = tf.nn.conv1d(x, W, s, padding='SAME')
        x = tf.nn.bias_add(x, b)
        return tf.nn.relu(x)

    if conv_layers == 2:
        def conv_net(x, weights, biases, dropout, activation_function):
            # Conv Layer 1
            conv1 = conv1d(x, weights['wc1'], biases['bc1'], 5)

            # Conv Layer 2
            conv2 = conv1d(conv1, weights['wc2'], biases['bc2'], 5)
    
            #Conv Layer 3
            # conv3 = conv1d(conv2, weights['wc3'], biases['bc3'], 1)
    

            # Fully connected layer 
            # Changed relu to sigmoid
            fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
            fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
            fc1 = activation_function(fc1)
            fc1 = tf.nn.dropout(fc1, dropout)

            # Output Layer - class prediction 
            out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
            return out
    elif conv_layers == 3:
        def conv_net(x, weights, biases, dropout, activation_function):
            # Conv Layer 1
            conv1 = conv1d(x, weights['wc1'], biases['bc1'], 5)

            # Conv Layer 2
            conv2 = conv1d(conv1, weights['wc2'], biases['bc2'], 5)
    
            #Conv Layer 3
            conv3 = conv1d(conv2, weights['wc3'], biases['bc3'], 1)
    

            # Fully connected layer 
            # Changed relu to sigmoid
            fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
            fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
            fc1 = activation_function(fc1)
            fc1 = tf.nn.dropout(fc1, dropout)

            # Output Layer - class prediction 
            out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
            return out

    x = tf.placeholder(tf.float32, [None, 250, 9])
    y = tf.placeholder(tf.float32, [None, n_classes])
    keep_prob = tf.placeholder(tf.float32)

    # Model
    logits = conv_net(x, weights, biases, keep_prob, activation_function)

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

    # Accuracy
    correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Launch the graph
    with tf.Session() as sess:
        sess.run(init)

        for epoch in range(epochs):
            sess.run(optimizer, feed_dict={
                x: f_train,
                y: l_train_one_hot.eval(),
                keep_prob: dropout})

            # Calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={
                x: f_train,
                y: l_train_one_hot.eval(),
                keep_prob: 1.})

            if epoch % 100 == 0:
                print('Epoch {:>2} - '
                    'Loss: {:>10.4f}'.format(
                    epoch + 1,
                    loss))

    # Calculate Test Accuracy
        test_acc = sess.run(accuracy, feed_dict={
            x: f_test,
            y: l_test_one_hot.eval(),
            keep_prob: 1.})
        print('Testing Accuracy: {}'.format(test_acc))

In [13]:
def transform_3d_2d(features):
    nsamples, nx, ny = features.shape
    return features.reshape((nsamples,nx*ny))

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
clf = RandomForestClassifier()
clf.fit(transform_3d_2d(f_train), l_train)
pred = clf.predict(transform_3d_2d(f_test))
test_acc = metrics.accuracy_score(l_test, pred)
print('Testing Accuracy: {}'.format(test_acc))

Testing Accuracy: 0.933026920551543


In [15]:
neural_net(tf.nn.sigmoid, 2, 0.0005, 2000, 0.8, 5, 123)

Epoch  1 - Loss:    43.7542
Epoch 101 - Loss:    10.6798
Epoch 201 - Loss:     6.4271
Epoch 301 - Loss:     5.5761
Epoch 401 - Loss:     4.6144
Epoch 501 - Loss:     4.3379
Epoch 601 - Loss:     3.9611
Epoch 701 - Loss:     3.7260
Epoch 801 - Loss:     3.5431
Epoch 901 - Loss:     3.8287
Epoch 1001 - Loss:     3.3900
Epoch 1101 - Loss:     3.1861
Epoch 1201 - Loss:     3.5413
Epoch 1301 - Loss:     3.1882
Epoch 1401 - Loss:     2.9227
Epoch 1501 - Loss:     2.9351
Epoch 1601 - Loss:     2.9868
Epoch 1701 - Loss:     3.0374
Epoch 1801 - Loss:     2.9358
Epoch 1901 - Loss:     2.7700
Testing Accuracy: 0.6605384349822998


In [16]:
neural_net(tf.nn.sigmoid, 2, 0.0005, 2000, 0.8, 5, 123)

Epoch  1 - Loss:    14.7844
Epoch 101 - Loss:    10.3681
Epoch 201 - Loss:     9.9865
Epoch 301 - Loss:     8.3634
Epoch 401 - Loss:     7.1704
Epoch 501 - Loss:     6.4957
Epoch 601 - Loss:     6.1401
Epoch 701 - Loss:     5.7386
Epoch 801 - Loss:     5.0314
Epoch 901 - Loss:     5.0131
Epoch 1001 - Loss:     4.4731
Epoch 1101 - Loss:     4.4357
Epoch 1201 - Loss:     3.7790
Epoch 1301 - Loss:     3.8026
Epoch 1401 - Loss:     3.9143
Epoch 1501 - Loss:     3.8617
Epoch 1601 - Loss:     3.6310
Epoch 1701 - Loss:     3.3495
Epoch 1801 - Loss:     3.3783
Epoch 1901 - Loss:     3.2074
Testing Accuracy: 0.6250820159912109
